# Topic Extraction in Newsfeeds
* Autor: Prof. Dr. Johannes Maucher
* Datum: 17.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)

# Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __RSS Feeds:__ Struktur von RSS Feeds analysieren und parsen mit dem _Universal Feed Parser_. 
* __Dokument Analyse:__ Die Häufigkeit aller Worte in einem Dokument (Inhalt des RSS Feeds) zählen und in einem Array verwalten. 
* __Merkmalsextraktion:__ Bestimmung von Merkmalen (hier auch: __Topics__) (Allgemein spricht man von Merkmalen. Im Fall, dass die NNMF auf Dokumente angewandt wird, werden die Merkmale auch mit __Topics__ oder __Themen__ bezeichnet) mit der \emph{Non Negative Matrix Factorization}.
* __Zuordnung__: Wie setzen sich die Topics aus den Wörtern zusammen? Wie stark sind die gefundenen Topics in den Artikeln vertreten?
* __Dokument Clustering:__ Mit der NNMF kann auch ein Clustering realisiert werden. Jeder Topic repräsentiert ein Cluster. Jedes Dokument wird dem Cluster zugeordnet, dessen Topic am stärksten in ihm vertreten ist. 

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Stellen Sie sich vor Sie möchten in eine eigene Webseite die RSS Feeds einer Menge von Nachrichtenservern einbinden. Da die unterschiedlichen Server wahrscheinlich Artikel zu den gleichen Themen anbieten, werden die Inhalte einiger Artikel ähnlich sein. Mit der __Nicht Negativen Matrixfaktorisierung (NNMF)__ kann für eine große Menge von Dokumenten eine Menge von Themen (Topics) ermittelt werden, auf die sich die Dokumente beziehen. Damit ist es u.a. möglich
* die Dokumente thematisch zu ordnen
* zu jedem Thema nur ein Dokument anzuzeigen

### Ähnlichkeiten bestimmen und relevante Merkmale extrahieren

Eine Sammlung von Dokumenten - in diesem Versuch die Menge aller Nachrichten der angegebenen Feeds - kann in einer Artikel/Wort-Matrix repräsentiert werden. Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile $i$, Spalte $j$ beschreibt wie häufig das Wort in Spalte $j$ im zur Zeile $i$ gehörenden Dokument vorkommt.

Unter der Annahme, dass Artikel umso ähnlicher sind, je mehr Worte in diesen gemeinsam vorkommen, kann auf der Grundlage dieser Matrix die Ähnlichkeit zwischen den Artikeln berechnet werden. Hierzu könnte die Matrix z.B. einfach einem _Hierarchischen Clustering_ übergeben werden. Das hierarchische Clustering weist jedoch im Fall einer großen Menge von zu vergleichenden Objekten zwei wesentliche Nachteile auf: Erstens ist die wiederholte Berechnung der Distanzen zwischen allen Artikeln/Clustern extrem rechenaufwendig, zweitens ist die Darstellung einer großen Anzahl von Objekten im Dendrogramm nicht mehr übersichtlich. 

Für das Auffinden von Assoziationen zwischen Dokumenten hat sich in den letzten Jahren die Methode der __Nicht-Negativen Matrix Faktorisierung (NNMF)__ etabliert. Mit dieser Methode kann eine Menge von wesentlichen Merkmalen berechnet werden, anhand derer sich die Dokumente clustern lassen, d.h. Dokumente des gleichen Clusters repräsentieren das gleiche Merkmal (Thema). Ein solches Merkmal wird durch eine Menge von Worten beschrieben, z.B. $\{$ _Paris, terror, IS_ $\}$  oder $\{$_refugee, syria, border_ $\}$. Neben der Merkmalsextraktion stellt die relativ geringe Komplexität einen weiteren Vorteil der NNMF dar. Durch die Darstellung der Artikel/Wort-Matrix als Produkt von 2 Faktormatrizen müssen deutlich weniger Einträge gespeichert werden.

### Nicht Negative Matrixfaktorisierung: Die Idee

Die Artikel/Wort-Matrix wird im Folgenden mit $A$ bezeichnet. Sie besitzt $r$ Zeilen und $c$ Spalten, wobei $r$ die Anzahl der Artikel und $c$ die Anzahl der relevanten Worte in der Menge aller Artikel ist. Durch Multiplikation der Matrix $A$ mit dem Vektor $v$ (_wordvec_: Vektor der alle relevanten Worte enthält) werden die Worte den Artikeln $a$ (_articletitles_: Vektor der alle Artikeltitel enthält) zugeordnet:

$$
a=A*v.
$$

Die Idee der NNMF besteht darin die Matrix $A$ als Produkt zweier Matrizen $W$ und $H$ darzustellen,

$$
A=W*H
$$

wobei alle Elemente in $W$ und $H$ größer oder gleich Null sein müssen. Die Matrixmultiplikation erfordert, dass die Anzahl der Zeilen $m$ in $H$ gleich der Anzahl der Spalten in $W$ sein muss. 
Durch die Faktorisierung der Matrix $A$ wird die Zuordnung der Wörter des Wortvektors $v$  zu den Artikeln des Vektors $a$ in zwei Stufen zerlegt. 

$$
f = H*v
$$
$$
a = W*f 
$$

In der ersten Stufe werden durch die Multiplikation von $v$ mit der Matrix $H$ die Wörter einem sogenannten Merkmalsvektor $f$ mit $m$ Elementen zugewiesen. In der zweiten Stufe werden durch die Multiplikation des Merkmalsvektor $f$ mit der Matrix $W$ die einzelnen Merkmale den Artikeln in $a$ zugeordnet. Die Matrix $H$ definiert also aus welchen Wörtern die Merkmale gebildet werden. Sie wird deshalb __Merkmalsmatrix__ genannt. Die Matrix $W$ hingegen beschreibt mit welchem Gewicht die einzelnen Merkmale in den verschiedenen Artikeln auftreten. Sie wird deshalb __Gewichtungsmatrix__ genannt.

Daraus folgt: Wenn eine Faktorisierung der Matrix $A$ gefunden wird, dann werden damit auch relevante Merkmale, also die Themen, definiert, hinsichtlich derer die Artikel effizient kategorisiert werden. Durch die Matrixfaktorisierung wird eine __Merkmalsextraktion__ realisiert. 

### Berechnung der Matrixfaktoren

Für die Berechnung der Faktoren wurde in [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) eine iterative Methode vorgestellt, die derzeit wohl am häufigsten angewandt wird und auch in dieser Übung implementiert werden soll. Der Algorithmus besteht aus folgenden Schritten:
* Gebe die zu faktorisierende Matrix $A$ ein. $r$ sei die Anzahl der Zeilen und $c$ die Anzahl der Spalten von $A$.
* Wähle die Anzahl $m$ der Merkmale, mit $m<c$. _Tipp:_ Für $m$ sollte zunächst ein Wert im Bereich $15$ bis $30$ gewählt werden.
* Lege eine $m \times c$ Matrix $H$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Lege eine $r \times m$ Matrix $W$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Wiederhole bis maximale Anzahl der Iteration erreicht oder Kosten $k$ unter vordefinierter Schwelle:

	* Berechne aktuelles Produkt $B=W*H$ und bereche die Kostenfunktion 
		$$
			k=\left\| A - B \right\|^2 = \sum\limits_{i,j} \left(A_{i,j} - B_{i,j}\right)^2
		$$ 
	* Anpassung der Matrix $H$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		H_{i,j} := H_{i,j} \frac{(W^T*A)_{i,j}}{(W^T*W*H)_{i,j}}
		$$
        
	* __Nach__ der Anpassung der Matrix $H$: Anpassung der Matrix $W$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		W_{l,i} := W_{l,i} \frac{(A*H^T)_{l,i}}{(W*H*H^T)_{l,i}}
		$$

In [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) ist bewiesen, dass durch die o.g. Anpassungsroutinen die Kosten $k$ monoton abnehmen und in einem Minimum konvergieren. Der Algorithmus ist jedoch nicht optimal weil das gefundene Minimum ein lokales Minimum sein kann.

 ## Vor dem Versuch zu klärende Fragen
 
 * Was versteht man unter Artikel/Wort-Matrix? Wie wird diese im aktuellen Versuch gebildet?

|      | Spalte   |  1.  |   2. |
| ---- |:-------------:| -----:|
|Zeile | Doclist | 'wort1' | 'wort2' |
|   1.  | Doc1 |  1   |  0   | 
|   2.  | Doc2 |  0   |  0   | 
|   3.  | Doc3 |  0   |  0   |  

Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile  i, Spalte  j beschreibt wie häufig das Wort in Spalte j im zur Zeile i gehörenden Dokument vorkommt.

 
Gebildet wird diese Matrix in dem man eine bereinigte Liste aus allen Wörtern zu jedem Dokument erstellt wird. Zur Bereinigung zählen: Stoppwörter, Doppelte Wörter und Satzzeichen zu entfernen (Tokenizing). Außerdem das Normalisieren der Wortmenge. Diese Listen werden in einer Liste gespeichert.
Anschließend wird aus diesem Objekt ein Gensim-Dictionary erzeugt. Es besitzt nun die Eigenschaft, dass es jedem relevanten Wort (Key) einen zufälligen, eindeutigen  Index (Value) erzeugt. 


Um die Performance zu erhöhen wird die Efficient Corpus Representation eingesetzt. Dabei wird aus dem Dictionary eine Liste aus Tupeln erzeugt, wobei der erste Wert, der Indize des Wortes ist und der zweite Wert die Häufigkeit des Wortes repräsentiert. Dies bildet den Corpus.

* Wie multipliziert man die Matrix
    $$
    A= \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
    $$
    mit dem Vektor  
    $$
    v=\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
    $$
    

So multipliziert man die Matrix:
    $$
    a = A * v = \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
     *    
\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
     =
\left( \begin{array}{c}
a_{00}*v_{0} + a_{01}*v_{1} + a_{02}*v_{2} + a_{03}*v_{3}  \\ 
a_{10}*v_{0} + a_{11}*v_{1} + a_{12}*v_{2} + a_{13}*v_{3} \\ 
a_{20}*v_{0} + a_{21}*v_{1} + a_{22}*v_{2} + a_{23}*v_{3} \\ 
a_{30}*v_{0} + a_{31}*v_{1} + a_{32}*v_{2} + a_{33}*v_{3}
\end{array} \right)
    $$
    
Die Voraussetzung das zwei Matritzen *A* , *v* miteinander multipliziert werden können ist das die Anzahl der spalten in *A* der Anzahl der Zeilen in *v* entspricht. Da in diesem Fall *v* ein Vektor ist, muss er genau so viele Dimensionen haben wie die Matrix Spalten.

* Was versteht man im Kontext der NNMF unter
    * Merkmalsmatrix
    * Gewichtsmatrix

* Wie werden in Numpy zwei Arrays (Typ numpy.array) 
	* im Sinne der Matrixmultiplikation miteinander multipliziert?
	* elementweise multipliziert?
* Wie wird die Transponierte eines Numpy-Arrays berechnet?

**Elementweise:**

`np.multiply(x1, x2)`

**Matrixmultiplikation:**

`np.matmul(a, b)`

**Transponierten:**

`np.transpose(x)`

# Versuchsdurchführung
Die in diesem Versuch einzubindenden Feeds sind in der unten stehenden Liste _feedlist_ definiert. Die aus dem vorigen Vesuch bereits bekannte Funktion _stripHTML()_ ist ebenfalls gegeben:

In [2]:
import feedparser
import numpy as np
import pandas as pd

feedlist=['http://feeds.reuters.com/reuters/topNews',
          'http://feeds.reuters.com/reuters/businessNews',
          'http://feeds.reuters.com/reuters/worldNews',
          'http://feeds2.feedburner.com/time/world',
          'http://feeds2.feedburner.com/time/business',
          'http://feeds2.feedburner.com/time/politics',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml',
          'http://feeds.nytimes.com/nyt/rss/Business',
          'http://www.nytimes.com/services/xml/rss/nyt/World.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/Economy.xml'
          ]

In [3]:
def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p

In [4]:
rssfeeds = pd.DataFrame(columns=['title','text','source'])

print "--------------------Articles------------------------"
for feed in feedlist:
    print "*"*30
    print feed
    f=feedparser.parse(feed)
    for e in f.entries:
      #print '\n---------------------------'
      if 'summary' in e.keys() and 'title' in e.keys():
        text = stripHTML(e.description)
        title = stripHTML(e.title)
        rssfeeds = rssfeeds.append(pd.DataFrame({'title': title, 'text': text, 'source': feed}, index=[0]), ignore_index=True)
print "------------------------------------------------------"
print "------------------------------------------------------"

--------------------Articles------------------------
******************************
http://feeds.reuters.com/reuters/topNews
******************************
http://feeds.reuters.com/reuters/businessNews
******************************
http://feeds.reuters.com/reuters/worldNews
******************************
http://feeds2.feedburner.com/time/world
******************************
http://feeds2.feedburner.com/time/business
******************************
http://feeds2.feedburner.com/time/politics
******************************
http://rss.cnn.com/rss/edition.rss
******************************
http://rss.cnn.com/rss/edition_world.rss
******************************
http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml
******************************
http://feeds.nytimes.com/nyt/rss/Business
******************************
http://www.nytimes.com/services/xml/rss/nyt/World.xml
******************************
http://www.nytimes.com/services/xml/rss/nyt/Economy.xml
--------------------------------

In [5]:
print rssfeeds.sample(5)

                                                source  \
124           http://rss.cnn.com/rss/edition_world.rss   
163  http://www.nytimes.com/services/xml/rss/nyt/Gl...   
36             http://feeds2.feedburner.com/time/world   
30             http://feeds2.feedburner.com/time/world   
101           http://rss.cnn.com/rss/edition_world.rss   

                                                  text  \
124  Hooded men broke into a shop in Paris and atte...   
163  Attendees worked side by side during a Quilt c...   
36   Previously, female fans could only watch sport...   
30   Eight suspected members of an online pedophile...   
101  President Donald Trump is about to come face-t...   

                                                 title  
124                                                     
163  Come on Over to My Place, Sister Girlfriend, a...  
36   Women in Saudi Arabia Can Finally Attend Live ...  
30   Mother Let Others Rape Her Son for Money as Pa...  
101  Opinion: Scr

## Anlegen der Artikel/Wort-Matrix

### Die Funktion _getarticlewords()_
Schreiben Sie eine Funktion _getarticlewords()_, die folgende Elemente zurückgibt:

* _allwords:_ ist ein Dictionary dessen Keys die Worte aller gesammelten Artikel sind. Der zu jedem Key gehörende Wert ist die Anzahl, wie oft das Wort insgesamt vorkommt.
* _articlewords:_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Listenelement ist ein Dictionary, welches die Worte des jeweiligen Artikels als Key enthält und als Wert die Worthäufigkeit.
* _articletitles_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Element ist der Artikeltitel als String.

Für das Parsing der Feeds soll wieder das Modul _feedparser_ eingesetzt werden. Die zu einer Nachricht gehörenden Wörter sollen die Wörter des Elements _title_ und die Wörter des Elements _description_ sein (siehe voriger Versuch). Allerdings sollen hier nicht alle Wörter eingebunden werden, sondern wie im vorigen Versuch eine Methode _getwords()_ implementiert werden, welche nur die _relevanten_ Wörter zurückgibt. Die Frage welche Wörter relevant sind ist nicht eindeutig beantwortbar. Sie können sich hierzu eigene Antworten einfallen lassen. Auf jeden Fall sollten aber die Stopwörter ignoriert werden. Hierzu kann z.B. die Stopwortliste von NLTK angewandt werden.

Nachdem alle relevanten Wörter aller Nachrichten gesammelt sind, sollte eine weitere Bereinigung stattfinden, die 

* alle Wörter, die weniger als 4 mal vorkommen
* alle Wörter, die in mehr als 30% aller Dokumente vorkommen

entfernt. 

Durch dieses Herausfiltern nicht relevanter Wörter kann es vorkommen, dass einzelne Artikel keine relevanten Wörter mehr enthalten. Diese Artikel sollen dann ganz ignoriert werden. D.h. unter anderem, dass diese Artikel auch nicht in _articlewords_ und _articletitles_ erscheinen.

In [6]:
from collections import defaultdict
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import ngrams

def to_dict(d):
    '''helper class – converts defaultdict to regular dict'''
    if isinstance(d, defaultdict):
        d = {k: to_dict(v) for k, v in d.iteritems()}
    return d

def getwords_adv(doc):
    
    #doc = doc.lower()
    
    #tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(doc)
    
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    #stemming
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    tokens = [stemmer.stem(word) for word in tokens]
    
    #add bigrams
    docsting = ' '.join(tokens).strip()
    n = 2
    n_grams = ngrams(docsting.split(), n)
    tokens = tokens + [' '.join(grams) for grams in n_grams]
    
    #remove words smaller than 4 and bigger than 60
    #tokens = [token for token in tokens if len(token) in range(4,60)]
    return tokens

def getarticlewords(articles):
    allwords = defaultdict(int)
    articletitles = []
    articlewords = []
    
    for index, row in articles.iterrows():
        title = row.loc['title']
        fulltext = row.loc['title'] + ' ' + row.loc['text']
        # add articletitles
        articletitles.append(title)
        # add articlewords
        words = getwords_adv(fulltext)
        wordsdict = defaultdict(int)
        for word in words: 
            wordsdict[word] += 1
            # extend allwords
            allwords[word] += 1
        articlewords.append(wordsdict)
    
    #remove words where count is lower 4
    wordsToRemove = []
    for word,count in allwords.items():
        if count < 4:
            #gather words to be removed
            wordsToRemove.append(word)
            #remove from allwords
            allwords.pop(word, None)
    
    #remove words from articlewords
    for words in articlewords:
        for word in wordsToRemove:
            words.pop(word, None)
    #remove word that are included in more than 30% of all documents
    
    return to_dict(allwords),articlewords,articletitles

allwords,articlewords,articletitles = getarticlewords(rssfeeds)
print len(allwords)
print len(articlewords)
print to_dict(articlewords[0])
print '\n',to_dict(allwords)
#print to_dict([to_dict(words) for words in articlewords])

[nltk_data] Downloading package stopwords to /home/marius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
491
277
{u'trump': 2, u'washington': 1, u'u s': 1, u'haiti african': 2, u'donald': 1, u'immigr': 1, u'presid donald': 1, u'home': 1, u'face': 1, u'unit': 1, u'condemn': 2, u'african countri': 2, u'use': 1, u'said': 1, u'describ': 1, u'would': 1, u'reuter': 1, u'bad': 1, u'state': 1, u'senat': 1, u'washington reuter': 1, u'forc': 1, u'peopl': 1, u'friday': 1, u'presid': 1, u'donald trump': 1, u'plan': 1, u'haiti': 2, u'abroad': 1, u'trump friday': 1, u'remark': 1, u'word': 1, u'critic': 1, u'unit state': 1, u's': 1, u'deni': 2, u'u': 1, u'african': 2, u'countri': 3, u'shithol': 1}

{u'last month': 5, u'founder': 5, u'global': 7, u'month': 9, u'protest': 4, u'go': 4, u'follow': 8, u'children': 5, u'thursday': 8, u'trump excel': 4, u'minist': 5, u'deal presid': 4, u'facebook': 22, u'govern': 7, u'administr': 4, u'die': 4, u'hous': 14, u'wednesday': 9, u'chang news'

### Die Funktion _makematrix()_
Schreiben Sie eine Funktion _makematrix()_, die aus dem Dictionary _allwords_ und der Liste _articlewords_ (vorige Aufgabe) die Artikel-/Wort-Matrix generiert. Die Einträge in der Matrix sollen die Häufigkeiten der Wörter im jeweiligen Dokument sein (term frequency tf). Die Artikel-/Wort-Matrix soll als 2-dimensionales Numpy Array angelegt werden.

In [7]:
from gensim import corpora, models, similarities

'''def makematrix(allwords, articlewords):
    
    dictionary = corpora.Dictionary(articlewords)
    v = dictionary.token2id
    print v
    #A = 
    print("Total number of documents in the dictionary: ",dictionary.num_docs)
    print("Total number of corpus positions: ",dictionary.num_pos)
    print("Total number of non-zeros in the BoW-Matrix: ",dictionary.num_nnz)
    print("Total number of different words in the dictionary: ",len(dictionary))'''
    
# makematrix(allwords, articlewords)

def get_row(all_words, article_words):
    for word, value in all_words.items():
        try:
            all_words[word] = article_words[word]
        except KeyError:
            value = 0
    article_array = np.array(all_words.values()) 
    return article_array

def makematrix(article_words, all_words):
    A = np.empty(shape=(0,len(all_words)), dtype=int) #<- hier stimmt was noch net
    for words_of_single_article in article_words:
        v = get_row(all_words, words_of_single_article)
        A = np.vstack((A, v))
    return A

matrix_A = makematrix(articlewords,allwords)
print matrix_A.shape
print matrix_A

(277, 491)
[[0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


## Die Nicht Negative Matrix Faktorisierung
Die Implementierung der NNMF ist entsprechend der Beschreibung im Theoriekapitel durchzuführen.

* Implementieren Sie die Funktion _cost(A,B)_. Dieser Funktion werden zwei Numpy-Matrizen $A$ und $B$ übergeben. Zurück geliefert werden die nach oben angegebener Formel berechneten Kosten $k$. Diese Funktion wird von der im folgenden beschriebenen Funktion _nnmf(A,m,it)_ benutzt.
* Implementieren Sie die Funktion __nnmf(A,m,it)__. In dieser Funktion soll der oben beschriebene Algorithmus für die Nicht-negative Matrix Faktorisierung ausgeführt werden. Der Funktion wird die zu faktorisierende Matrix $A$, die Anzahl der Merkmale $m$ und die Anzahl der Iterationen $it$ übergeben. Die Funktion gibt die gefundenen Faktoren $W$ und $H$ zurück. In jeder Iteration sollen mit der Funktion __cost(A,B)__ die Kosten berechnet werden. Sobald die Kostenabnahme pro 10 Iterationen kleiner als $2$ ist oder eine maximale Anzahl von Iterationen ($maxIt=200$) erreicht ist, soll der Algorithmus mit der Rückgabe der Faktoren $W$ und $H$ terminieren.     


Tipp für die Implementierung elementweiser Operationen von Matrizen: Für elementweise Operationen müssen in Python/Numpy nicht alle Elemente über Schleifen explizit berechnet werden. Eine elementweise Anpassung aller Matrixelemente kann kompakt programmiert werden indem die beteiligten Matrizen für diese Operationen als Arrays implementiert werden. Sollen z.B. die beiden gleich großen Numpy Arrays $U$ und $V$ elementweise multipliziert werden, dann wäre der entsprechende Programmcode einfach _U*V_.  

In [71]:
# Erstellen der Matrix H und W
def generate_random_matrix(rows, columns):
    return np.random.random((rows, columns))

count_of_features = 15 # (m) Merkmale
row_count_matrix_A = len(matrix_A)
#print row_count_matrix_A
col_count_matrix_A = len(matrix_A[2])

matrix_H = generate_random_matrix(count_of_features,col_count_matrix_A)
matrix_W = generate_random_matrix(row_count_matrix_A, count_of_features)

# Erstellen der Matrix B, mit B = W * H

matrix_B = np.matmul(matrix_W, matrix_H)

#print matrix_B

# Berechnung Kostenfaktor k

def calc_cost(matrix_A, matrix_B):
    sub_matrix = np.subtract(matrix_A, matrix_B)
    sum_of_all = 0
    for v in sub_matrix:
        for v_2 in v:
            sum_of_all += v_2*v_2
    return sum_of_all

#print calc_cost(matrix_A, matrix_B)

def adapt_H(A, H, W):
    W_t = np.transpose(W)
    nummerator = np.matmul(W_t,A)
    denominator = np.matmul(np.matmul(W_t,W), H)
    fract = np.divide(nummerator,denominator)
    result = H*fract # fraglich
    return result
    
def adapt_W(A, H, W):
    H_t = np.transpose(H)
    nummerator = np.matmul(A, H_t)
    denominator = np.matmul(np.matmul(W,H), H_t)
    fract = np.divide(nummerator,denominator)
    result = W*fract # fraglich
    return result
    
def nnmf(A,m,it):
    H = generate_random_matrix(count_of_features,col_count_matrix_A)
    W = generate_random_matrix(row_count_matrix_A, count_of_features)    
    cost_list = []
    for i in range(it):
        H = adapt_H(A, H, W)
        B = np.matmul(W, H)
        cost_list.append(calc_cost(A, B))
        if len(cost_list) > 10 and (cost_list[-10] - cost_list[-1]) < 2.0:
            break
    W = adapt_W(A, H, W)
    return (i, cost_list,H,W)

interations, costs, matrix_H, matrix_W = nnmf(matrix_A, 30, 200)

print interations, costs, '\n','\n','\n',  matrix_H, matrix_W

32 [4604.3327277225999, 4595.014293419491, 4587.0674655182229, 4580.3082138982209, 4574.5825483815561, 4569.7458479466432, 4565.6637030820175, 4562.2156016511344, 4559.2971235116302, 4556.8198924377475, 4554.7100739818752, 4552.9063394599361, 4551.3578827385336, 4550.022723955668, 4548.8662975250818, 4547.8602425380732, 4546.9813396735944, 4546.2105802421738, 4545.5323671801461, 4544.9338434459432, 4544.4043375162037, 4543.9349138769503, 4543.5180164078156, 4543.1471920078648, 4542.8168806360409, 4542.522257384333, 4542.2591131540848, 4542.0237628012519, 4541.8129724827704, 4541.6239005378193, 4541.4540481820659, 4541.3012175553431, 4541.1634754420811] 


[[  8.21093688e-03   7.37982334e-07   1.24872240e-02 ...,   1.07095681e-05
    2.80634302e-02   1.47722961e-03]
 [  6.63294915e-11   7.45894194e-06   2.96207605e-03 ...,   3.74185197e-05
    1.04042584e-04   1.96852276e-06]
 [  2.23113305e-05   3.22215654e-02   7.94366458e-03 ...,   7.11853012e-02
    1.90826421e-12   4.00180254e-04]


## Anzeige der Merkmale und der Gewichte

Im vorigen Abschnitt wurde die Merkmalsmatrix $H$ und die Gewichtsmatrix $W$ berechnet. Diese Matrizen können natürlich am Bildschirm ausgegeben werden, was jedoch nicht besonders informativ ist. Aus den Matrizen können jedoch die Antworten für die folgenden interessanten Fragen berechnet werden:

* In welchen Artikeln sind welche Merkmale stark vertreten?
* Wie lassen sich die insgesamt $m$ Merkmale beschreiben, so dass aus dieser Merkmalsbeschreibung klar wird, welches Thema den Artikeln, in denen das Merkmal stark vertreten ist, behandelt wird? 
 
Die Antwort auf die erste Frage ergibt sich aus der Gewichtsmatrix $W$. Für die Beantwortung der zweiten Frage wird die Merkmalsmatrix $H$ herangezogen.



### Beschreibung der Merkmale

Die Merkmalsmatrix $H$ beschreibt, wie stark die Worte aus _wordvec_ in jedem Merkmal enthalten sind. Jede Zeile von $H$ gehört zu einem Merkmal, jede Spalte von $H$ gehört zu einem Wort in _wordvec_.

Es bietet sich an jedes Merkmal einfach durch die $N=6$ Wörter aus _wordvec_ zu beschreiben, welche am stärksten in diesem Merkmal enthalten sind. Hierzu muss für jedes Merkmal die entsprechende Zeile in $H$ nach den $N=6$ größten Werten durchsucht bzw. geordnet werden. Die entsprechenden Spalten dieser Matrixelemente verweisen dann auf die $N=6$ wichtigsten Worte des Merkmals.

Tipp für die Implementierung: Legen Sie für jedes Merkmal $i$ eine Liste an. Die Listenlänge ist durch die Anzahl der Worte in _wordvec_ (d.h. die Anzahl der Spalten in $H$) gegeben. Jedes Listenelement $j$ enthält selbst wieder 2 Elemente: An erster Stelle den entsprechenden Wert $H_{i,j}$ der Merkmalsmatrix, an der zweiten Stelle das $j.$-te Wort in _wordvec_. Nachdem die Liste angelegt ist, kann sie mit _listname.sort()_ in aufsteigender Reihenfolge sortiert werden. Die abnehmende Sortierung erhält man mit _listname.sort().reverse()_. Danach geben die $N=6$ ersten Listenelemente die für das Merkmal $i$ wichtigsten Worte an.

   
### Präsenz der Merkmale in den Artikeln

Die Gewichtsmatrix $W$ beschreibt, wie stark die $m$ Merkmale in den Artikeln aus _articletitles_ enthalten sind. Jede Zeile von $W$ gehört zu einem Artikel, jede Spalte von $W$ gehört zu einem Merkmal.
Die Berechnung der $M=2$ gewichtigsten Merkmale für jeden Artikel in _articletitles_ kann analog zu der oben beschriebenen Berechnung der $N=6$ wichtigsten Worte eines Merkmals berechnet werden.


### Implementierung

Implementieren Sie eine Funktion _showfeatures(w,h,titles,wordvec)_, welche wie oben beschrieben für jeden Artikel die $M=2$ wichtigsten Merkmale am Bildschirm ausgibt. Dabei soll jedes Merkmal durch die 6 wichtigsten Wörter dieses Merkmals angegeben werden. Siehe Beispielausgabe unten.  

Übergabeparameter der Funktion sind die Merkmalsmatrix $H$, die Gewichtungsmatrix $W$, die Liste aller Artikeltitel _articletitles_ und die Liste aller Worte _wordvec_.


Beispiel fuer Ausgabe:

[(13.54131155883748, 13, u'Putin vows payback after confirmation of Egypt plane bomb'),

(2.2466669548146254, 9, u'Putin vows payback after confirmation of Egypt plane bomb')]

----- ['plane', 'egypt', 'russia', 'month', 'killing', 'putin']

----- ['airport', 'russian', 'crash', 'egypt', 'security', 'officials']

Die Ausgabe ist wie folgt zu interpretieren:
* Für den Artikel _Putin vows payback after confirmation of Egypt plane bomb_ ist 
    * das wichtigste Merkmal durch die 6 Wörter _plane_, _egypt_, _russia_, _month_, _killing_, _putin_ definiert. Das Gewicht dieses Merkmals im Artikel ist 13.54
    * das zweitwichtigste Merkmal durch die 6 Wörter _airport_, _russian_, _crash_, _egypt_, _security_, _officials_ definiert. Das Gewicht dieses Merkmals im Artikel ist 2.24

In [100]:
# Beschreibung der Merkmale

def get_the_biggest(indexed_features, n):
    sorted_list = sorted(indexed_features, key=lambda value: value[1])
    biggest = []
    for i in range(-(n+1),-1):
        biggest.append(sorted_list[i])
    return biggest

def get_index_list(feature):
    indexed_list = []
    index = 0
    for value in feature:
        indexed_list.append((index,value))
        index +=1
    return indexed_list

def find_important_features(matrix, n):
    important_feature_list = []
    for feature in matrix:
        indexed_list = get_index_list(feature)
        biggest_six = get_the_biggest(indexed_list, n)
        important_feature_list.append(biggest_six)
    return important_feature_list


print find_important_features(matrix_H, 6)
print find_important_features(matrix_W, 2)


# Implementierung
def showfeatures(W,H,titles,wordvec):
    # TODO
    return 0

print showfeatures(matrix_W,matrix_H,articletitles,allwords)


[[(6, 0.055627528043210549), (42, 0.059762686749649144), (136, 0.062927918657517207), (30, 0.064483897448640234), (34, 0.079518751089576553), (353, 0.089829012488288201)], [(205, 0.034787080603170088), (199, 0.036991366544601605), (393, 0.052741678063339292), (237, 0.053192518063817848), (128, 0.075994579023961797), (303, 0.076455664313787614)], [(16, 0.051764254696896599), (293, 0.054822368406976719), (307, 0.056374429337523145), (298, 0.057746242236197808), (488, 0.071185301234281087), (34, 0.080968139805558489)], [(472, 0.053859531196461134), (488, 0.0560569102070946), (38, 0.066371733508431466), (204, 0.067529442579129129), (12, 0.068193154205559714), (79, 0.086817363750032003)], [(130, 0.023818270388038271), (40, 0.024268370900184411), (268, 0.028904594031264455), (90, 0.031076918037640778), (431, 0.038056555647777539), (34, 0.084178960264213537)], [(317, 0.037135683541428217), (353, 0.047679193749597358), (142, 0.047943596439308636), (370, 0.051833862535535333), (111, 0.053444784

In [49]:
# Diese Methoden dienen nur zu lesbarkeit des Codes
def calc_a(A, v):
    return np.matmul(A,v)

def calc_f(H, v):
    return np.matmul(W,v)

def calc_a(W, f):
    return np.matmul(W,v)

In [ ]:
#Your code

## Aufgaben

1. Analysieren Sie die berechneten Topics indem Sie sich überlegen ob die gefundenen 6 Wörter pro Topic wirklich Themen beschreiben.
2. Verändern Sie die Parameter der NNMF (Anzahl der Topics $m$, Anzahl der Iterationen). Bei welcher Einstellung der Parameter erhalten Sie das für sie sinnvollste Resultat (sinnvolle Topics)?
3. Wie kann die _getwords()_ Methode verbessert werden, so dass noch bedeutsamere Topics gefunden werden? 

In [ ]:
#Your markdown